# Part 1: Sequential Conversation 

### Imports & API Keys

In [16]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, ChatPromptTemplate

from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, SystemMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

In [5]:
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "chatbot_agent"

### Initialize LLM model

In [22]:
# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model="gpt-4.1-nano")

### Prompt Template

In [10]:
system_prompt = "You are a helpful assistant called Alex."

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}"),
])

### Chatbot Chain

In [23]:
pipeline = prompt_template | llm

### ConversationSummaryBufferMemory
Based on number of messages. Where if number of messages is more than k, pop oldest messages and create a new summary by adding information from poped messages.

In [40]:
class ConversationSummaryBufferMemory_custom(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing ConversationSummaryBufferMemory_custom with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """
        Add messages to the history, removing any messages beyond the last 'k' messages.
        """
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

# initialize memory
chat_map = {}

# function to get memory for specific session id
def get_chat_history(session_id: str, k: int = 4) -> ConversationSummaryBufferMemory_custom:
    print(f"get_chat_history called with session_id={session_id} and k={k}")
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryBufferMemory_custom(k=k)
    # remove anything beyond the last
    return chat_map[session_id]

### RunnableWithMessageHistory

In [42]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key = "query",
    history_messages_key= "chat_history",
    history_factory_config= [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=10,
        )
    ]
)

### Test LLM Chat with history

In [43]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"configurable": {"session_id": "id_k10", "k": 10}}
)

get_chat_history called with session_id=id_k10 and k=10
Initializing ConversationSummaryBufferMemory_custom with k=10


AIMessage(content='Hello, James! Nice to meet you. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 25, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_950f36939b', 'finish_reason': 'stop', 'logprobs': None}, id='run--35894b97-d6a2-4914-8e87-958ff658ba2b-0', usage_metadata={'input_tokens': 25, 'output_tokens': 16, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [44]:
chat_map["id_k10"].clear()  # clear the history

# manually insert history
chat_map["id_k10"].add_user_message("Hi, my name is James")
chat_map["id_k10"].add_ai_message("I'm an AI model called Alex.")
chat_map["id_k10"].add_user_message("I'm researching the different types of conversational memory.")
chat_map["id_k10"].add_ai_message("That's interesting, what are some examples?")
chat_map["id_k10"].add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
chat_map["id_k10"].add_ai_message("That's interesting, what's the difference?")
chat_map["id_k10"].add_user_message("Buffer memory just stores the entire conversation, right?")
chat_map["id_k10"].add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
chat_map["id_k10"].add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
chat_map["id_k10"].add_ai_message("Very cool!")

chat_map["id_k10"].messages

[HumanMessage(content='Hi, my name is James', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm an AI model called Alex.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Buffer window memory stores t

In [46]:
pipeline_with_history.invoke(
    {"query": "what is my name again?"},
    config={"configurable": {"session_id": "id_k10", "k": 10}}
)

get_chat_history called with session_id=id_k10 and k=10


AIMessage(content='Your name is James.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 152, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1a97b5aa6c', 'finish_reason': 'stop', 'logprobs': None}, id='run--00675b68-efbc-4d81-8132-1bb616d28772-0', usage_metadata={'input_tokens': 152, 'output_tokens': 5, 'total_tokens': 157, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})